In [ ]:
import matplotlib.pyplot as plt
import numpy as np

%load_ext autoreload
%aimport kalman_filter

In [ ]:
def fquad(A, B, C, tvals):
    return A + B*tvals + C * tvals**2

In [ ]:
def make_column(x0, xL, xQ, xstd, tvals):
    return fquad(x0, xL, xQ, tvals) + np.random.randn(tvals.shape[0]) * xstd


In [ ]:
def make_data(timestart, timestop, deltat, tstd,  x0, xL, xQ, xstd, y0, yL, yQ, ystd):
    tvals = np.arange(timestart, timestop, deltat)
    xvals = make_column(x0, xL, xQ, xstd, tvals)
    yvals = make_column(y0, yL, yQ, ystd, tvals)
    tvals += tvals + np.abs(np.random.randn(tvals.shape[0])) * tstd
    return np.concatenate([tvals, xvals, yvals]).reshape(3, -1).T


In [ ]:
timestart = 0
timestop = 10
deltat = 0.25
tstd = 0.1 * deltat

x0 = 200
xL = 5
xQ = 0.25
xstd = 0.1

y0 = 600
yL = -2.5
yQ = 0.2
ystd = 0.1


dataset = make_data(timestart, timestop, deltat, tstd, x0, xL, xQ, xstd, y0, yL, yQ, ystd)

In [ ]:
measurement_error = np.array(
    [
        xstd**2,
        ystd**2,
        np.sqrt(xstd**2 + tstd**2),
        np.sqrt(ystd**2 + tstd**2),
    ]
)

initial_state = np.zeros(3)

state_history = np.array([])
kf = kalman_filter.Kalman2D(measurement_error)
for datum in dataset:
    kf.update(datum)

    current_state = np.concatenate(
        [kf.get_time(), kf.get_position(), kf.get_velocity()]
    )

    # print(
    #     f"{kf.get_time().shape}, {kf.get_position().shape}, {kf.get_velocity().shape} {current_state.shape} {state_history.shape}"
    # )

    if state_history.shape[0] == 0:
        state_history = current_state
    else:
        state_history = np.concatenate([state_history, current_state])

state_history = state_history.reshape(-1, 5)


In [ ]:
plt.plot(dataset[:,0], dataset[:,1], 'o', label='x input')
plt.plot(state_history[:,0], state_history[:,1], '+', label="x fit")
plt.plot(dataset[:,0], dataset[:,2], '2', label='y input')
plt.plot(state_history[:,0], state_history[:,2], 'x', label="y fit")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
plt.plot(dataset[:,1], dataset[:,2], '3', label="input")
plt.plot(state_history[:,1], state_history[:,2], '4', label="fit")
plt.grid(True)
plt.legend()
plt.show()